### A08 - Bootstrapping

Utiliza los conceptos aprendidos en los laboratorios de regresión y clasificación para encontrar el error estándar de los coeficientes de una regresión (lineal/logística) simple para los datasets de “Advertising” y “Default”.

Utiliza bootstrap para simular 1000 remuestreos de esos datasets y calcula la media de los coeficientes obtenidos al aplicarle regresión a cada remuestreo. Calcula la desviación estándar.

Compara los resultados obtenidos con el método visto en los laboratorios contra los resultados obtenidos con bootstrap. ¿Por qué podría haber diferencias en los resultados?

Agrega regularización L2 a los modelos del dataset de Advertising (optimiza el hiperparámetro). Utiliza ese valor del hiperparámetro para repetir el experimento de los 1000 remuestreos. Calcula la desviación estándar de los coeficientes obtenidos.

### Advertising

In [2]:
# Cargamos el dataset de Advertising
import pandas as pd
adv = pd.read_csv("Advertising.csv")
adv.head()

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


In [3]:
import numpy as np
ADV = adv.drop(columns=['Unnamed: 0'])
# X = [1, TV, radio, newspaper]
y = ADV['sales']
X = ADV.drop(columns=['sales'])

In [4]:
# Utilizamos bootstrap para simular 1000 remuestreos del dataset
from sklearn.utils import resample
from sklearn.linear_model import LinearRegression

n_iterations = 1000
coeffs = []
for i in range(n_iterations):
    X_resampled, y_resampled = resample(X, y, replace=True)
    model = LinearRegression()
    model.fit(X_resampled, y_resampled)
    coeffs.append(model.coef_)
coeffs = np.array(coeffs)

# Calculamos la media y desviación estándar de los coeficientes obtenidos
coeffs_mean = np.mean(coeffs, axis=0)
coeffs_std = np.std(coeffs, axis=0)
print("Media de los coeficientes:", coeffs_mean)
print("Desviación estándar de los coeficientes:", coeffs_std)


Media de los coeficientes: [ 0.04568013  0.18906323 -0.00127234]
Desviación estándar de los coeficientes: [0.00190473 0.01095326 0.00618385]


### Default

In [5]:
# Cargamos el dataset de Default
default = pd.read_csv("Default.csv")
default.head()

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138950
3,No,No,529.250605,35704.493940
4,No,No,785.655883,38463.495880


In [6]:
# Convertir 'default' y 'student' a binaria (Yes/No a 1/0)
from sklearn.preprocessing import StandardScaler

default['default'] = default['default'].map({'Yes': 1, 'No': 0})

default['student'] = default['student'].map({'Yes': 1, 'No': 0})

# Separar variables predictoras y target
X = default.drop('default', axis=1)
y = default['default']

# Escalar características numéricas
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [7]:
# SIn Bootsrapping
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X,y)
coeffs = np.array(coeffs)
coeffs_mean = np.mean(coeffs, axis=0)
coeffs_std = np.std(coeffs, axis=0)
print("Media de los coeficientes:", coeffs_mean)
print("Desviación estándar de los coeficientes:", coeffs_std)


Media de los coeficientes: [ 0.04568013  0.18906323 -0.00127234]
Desviación estándar de los coeficientes: [0.00190473 0.01095326 0.00618385]


In [8]:
# Utilizamos bootstrap para simular 1000 remuestreos del dataset (en este caso regresion logistica)
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
n_iterations = 1000
coeffs = []
for i in range(n_iterations):
    X_resampled, y_resampled = resample(X_scaled, y, replace=True)
    model = LogisticRegression()
    model.fit(X_resampled, y_resampled)
    coeffs.append(model.coef_)
coeffs = np.array(coeffs)
# Calculamos la media y desviación estándar de los coeficientes obtenidos
coeffs_mean = np.mean(coeffs, axis=0)
coeffs_std = np.std(coeffs, axis=0)
print("Media de los coeficientes:", coeffs_mean)
print("Desviación estándar de los coeficientes:", coeffs_std)

Media de los coeficientes: [[-0.28807085  2.7511122   0.0432529 ]]
Desviación estándar de los coeficientes: [[0.10971733 0.11164363 0.1089977 ]]


### Interpretacion

Los resultados del método tradicional y del método bootstrap difieren porque el método clásico se basa en fórmulas teóricas que dependen de varios supuestos estadísticos (normalidad, linealidad, homocedasticidad). En cambio, el bootstrap estima la variabilidad utilizando remuestreo directo de los datos, sin depender de estos supuestos.

Esto hace que el bootstrap capture mejor la variabilidad real de los coeficientes, especialmente cuando el dataset es pequeño, ruidoso o cuando los supuestos teóricos no se cumplen perfectamente.

### Regularización

Agrega regularización L2 a los modelos del dataset de Advertising (optimiza el hiperparámetro). Utiliza ese valor del hiperparámetro para repetir el experimento de los 1000 remuestreos. Calcula la desviación estándar de los coeficientes obtenidos.

In [9]:
# Con penalización
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

y = ADV['sales'].values
X = ADV.drop(columns=['sales']).values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

params = {'alpha': np.logspace(-3, 3, 50)}
L2 = Ridge()
gs = GridSearchCV(L2, params, cv=5)
gs.fit(X_scaled, y)

best_alpha = gs.best_params_['alpha']
print("Mejor lambda (alpha):", best_alpha)

Mejor lambda (alpha): 1.151395399326447


In [10]:
L2_opt = Ridge(alpha=best_alpha).fit(X_scaled, y)

coef_ridge = L2_opt.coef_
intercept_ridge = L2_opt.intercept_

print("Coeficientes Ridge:", coef_ridge)
print("Intercepto Ridge:", intercept_ridge)

Coeficientes Ridge: [ 3.89734714  2.7746345  -0.01503966]
Intercepto Ridge: 14.0225


In [11]:
n_sim = 1000
n = len(X_scaled)
coef_ridge_boot = []

for b in range(n_sim):
    X_boot, y_boot = resample(X_scaled, y)
    ridge_b = Ridge(alpha=best_alpha).fit(X_boot, y_boot)

    params_b = np.concatenate(([ridge_b.intercept_], ridge_b.coef_))
    coef_ridge_boot.append(params_b)

coef_ridge_boot = np.array(coef_ridge_boot)
ridge_boot_mean = coef_ridge_boot.mean(axis=0)
ridge_boot_std = coef_ridge_boot.std(axis=0)

print("Media bootstrap L2 :", ridge_boot_mean)
print("STD bootstrap L2 :", ridge_boot_std)

Media bootstrap L2 : [ 1.40183288e+01  3.89690386e+00  2.77095518e+00 -1.39153578e-02]
STD bootstrap L2 : [0.12068229 0.16292665 0.16302721 0.13559291]


### Interpretacion

El uso de Ridge con penalización alpha : 1.15 y remuestreo bootstrap muestra que los coeficientes del modelo son estables y que la regularización ayuda a reducir la varianza del estimador.
Los predictores TV y radio mantienen un efecto positivo importante en las ventas, mientras que newspaper es prácticamente irrelevante, y Ridge correctamente lo penaliza acercándolo a cero.
El bootstrap confirma que los coeficientes presentan baja variabilidad, lo cual indica que el modelo es robusto frente al remuestreo de los datos.”